## Data Enrichment with AI

### Setup

##### Create a dataset for storing the AI tables and another for storing the AI models

In [ ]:
%%bigquery
CREATE SCHEMA `shidcs329e`.magazine_recipes_stg_ai

Query is running:   0%|          |

""


In [ ]:
%%bigquery
CREATE SCHEMA `shidcs329e`.remote_models

Query is running:   0%|          |

""


##### Before running this cell, create the remote connection and assign the IAM role `Vertex AI User` to the service account associated with the connection.

In [ ]:
%%bigquery
create or replace model remote_models.gemini_pro
  remote with connection `projects/shidcs329e/locations/us/connections/vertexconnection`
  options (endpoint = 'gemini-pro');

Query is running:   0%|          |

""


# Change 1: Detect Recipe 'TYPE' within Recipes table

In [ ]:
%%bigquery
select *
from magazine_recipes_stg.Recipes
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,recipe_id,title,subtitle,servings,yield_unit,prep_min,cook_min,stnd_min,source,intro,...,rating,ease_of_prep,note,type,page,slowcooker,link,last_made,data_source,load_time
0,1471,None,None,<NA>,None,<NA>,<NA>,<NA>,None,None,...,<NA>,None,None,None,<NA>,None,None,NaT,bird,2024-01-30 01:08:47.612652+00:00
1,1559,None,None,<NA>,None,<NA>,<NA>,<NA>,None,None,...,<NA>,None,None,None,<NA>,None,None,NaT,bird,2024-01-30 01:08:47.612652+00:00
2,1509,None,None,<NA>,None,<NA>,<NA>,<NA>,None,None,...,<NA>,None,None,None,<NA>,None,None,NaT,bird,2024-01-30 01:08:47.612652+00:00
3,1458,None,None,<NA>,None,<NA>,<NA>,<NA>,None,None,...,<NA>,None,None,None,<NA>,None,None,NaT,bird,2024-01-30 01:08:47.612652+00:00
4,1567,None,None,<NA>,None,<NA>,<NA>,<NA>,None,None,...,<NA>,None,None,None,<NA>,None,None,NaT,bird,2024-01-30 01:08:47.612652+00:00


In [ ]:
# there appear to be many null records, the LLM will not be able to predict if there is not data
%%bigquery
select count(recipe_id)
from magazine_recipes_stg.Recipes
where title is null

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,153


In [ ]:
%%bigquery
select count(quantity_id) from magazine_recipes_stg.Quantity
where recipe_id in (select recipe_id from magazine_recipes_stg.Recipes where title is null)

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,0


In [ ]:
# the null records are not in the junction table so there will not be orphan records if we delete them
%%bigquery
create or replace table magazine_recipes_stg.Recipes as
select * from magazine_recipes_stg.Recipes
where title is not null

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from magazine_recipes_stg.Recipes
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,recipe_id,title,subtitle,servings,yield_unit,prep_min,cook_min,stnd_min,source,intro,...,rating,ease_of_prep,note,type,page,slowcooker,link,last_made,data_source,load_time
0,143,Prosciutto apple flatbread pizza,None,<NA>,None,45,<NA>,<NA>,Sheet Pan Suppers,None,...,3,Fairly Easy,None,Main Dish,<NA>,None,None,2018-09-23,airtable,2024-02-02 21:09:57.475069+00:00
1,141,Soy Mustard Salmon,None,<NA>,None,20,<NA>,<NA>,Sheet Pan Recipes,None,...,4,Fairly Easy,None,Main Dish,84,None,None,2018-06-16,airtable,2024-02-02 21:09:57.475069+00:00
2,139,Tomato Basil Soup,None,<NA>,None,45,<NA>,<NA>,Good Housekeeping,None,...,4,Average,None,Main Dish,60,None,None,2019-01-12,airtable,2024-02-02 21:09:57.475069+00:00
3,142,Dill Cucumber Salmon,None,<NA>,None,30,<NA>,<NA>,Sheet Pan Suppers,None,...,4,Average,None,Main Dish,<NA>,None,None,2018-11-02,airtable,2024-02-02 21:09:57.475069+00:00
4,145,Scones,None,<NA>,None,45,<NA>,<NA>,"Pinterest,Taste of Home",None,...,5,Super Simple,Make dough night before,Baking Basics,<NA>,None,None,2017-05-13,airtable,2024-02-02 21:09:57.475069+00:00


In [ ]:
%%bigquery
SELECT column_name, COUNT(1) AS nulls_count
FROM magazine_recipes_stg.Recipes,
UNNEST(REGEXP_EXTRACT_ALL(TO_JSON_STRING('magazine_recipes_stg.Recipes'), r'"(\w+)":null')) column_name
GROUP BY column_name
ORDER BY nulls_count DESC

Query is running:   0%|          |

Downloading: |          |

,column_name,nulls_count


In [ ]:
# Some Recipes already have a Type, other recipes do not!
%%bigquery
select distinct(type) from magazine_recipes_stg.Recipes

Query is running:   0%|          |

Downloading:   0%|          |

,type
0,Main Dish
1,Baking Basics
2,Side Dish
3,None
4,Drink
5,Dessert
6,Salad
7,Breakfast


In [ ]:
# We don't really like the type 'Baking Basics'
%%bigquery
select title from magazine_recipes_stg.Recipes
where type = 'Baking Basics'

Query is running:   0%|          |

Downloading:   0%|          |

,title
0,Scones
1,Brioche Chocolate Rolls
2,Chocolate Chip Irish Soda Bread
3,Dutch Oven Bread
4,Naan
5,Baking Powder Biscuits
6,Lemon Poppy Seed Scones
7,Malteese Gilatti
8,Pretzel Sticks
9,Sugar Cookies


In [ ]:
# Baking Basics is not that descriptive of a cateogry
#  most could just be recategorized as desserts or side dishes, so we will set the category to null for these records and let the LLM recategorize them
%%bigquery
update magazine_recipes_stg.Recipes
set type = null
where type = 'Baking Basics'

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select distinct(type), count(*) from magazine_recipes_stg.Recipes
group by type

Query is running:   0%|          |

Downloading:   0%|          |

,type,f0_
0,None,892
1,Main Dish,102
2,Side Dish,9
3,Dessert,10
4,Drink,2
5,Salad,3
6,Breakfast,5


In [ ]:
# category that would be useful but are not present: soup, appetizer

In [ ]:
%%bigquery
declare prompt_query STRING default "Classify each recipe into one of these 8 categories (Main Dish, Side Dish, Dessert, Drink, Salad, Breakfast, Soup, Appetizer) with similar recipes in each category if the record does not already have a type. Assign each recipe to only one category. Return the output as json, include the recipe_id, title, and cateogry in the output";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "type", type))) as prompt
    from magazine_recipes_stg.Recipes
    order by recipe_id
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"{""recipe_id"":1,""title"":""Chive Butter Radishes""...",None,,Classify each recipe into one of these 8 categ...
1,"```json\n[\n {\n ""recipe_id"": 1,\n ""tit...",None,,Classify each recipe into one of these 8 categ...
2,"```json\n[\n {\n ""recipe_id"": 1,\n ""tit...",None,,Classify each recipe into one of these 8 categ...
3,"{""recipe_id"":4,""title"":""Balsamic Pork Chops"",""...",None,,Classify each recipe into one of these 8 categ...
4,"{""recipe_id"":1,""title"":""Chicken Noodle Soup"",""...",None,,Classify each recipe into one of these 8 categ...
5,"```json\n[\n {\n ""recipe_id"": 1,\n ""tit...",None,,Classify each recipe into one of these 8 categ...
6,"```json\n[\n {\n ""recipe_id"": 1,\n ""tit...",None,,Classify each recipe into one of these 8 categ...
7,"```json\n[\n {\n ""recipe_id"": 1,\n ""tit...",None,,Classify each recipe into one of these 8 categ...
8,"```json\n[\n {\n ""recipe_id"": 1,\n ""tit...",None,,Classify each recipe into one of these 8 categ...
9,"```json\n[\n {\n ""recipe_id"": 1,\n ""tit...",None,,Classify each recipe into one of these 8 categ...


In [ ]:
# we misunderstood the fact that the model would be fed one record at a time, after reading the prompt it was given, we can rewrite a better prompt
%%bigquery
declare prompt_query STRING default "Classify the recipe into one of these types (Main Dish, Side Dish, Dessert, Drink, Salad, Breakfast, Soup, Appetizer) if it does not aready have a type.  Return the output as json, include the recipe_id, title, and type";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "type", type))) as prompt
    from magazine_recipes_stg.Recipes
    order by recipe_id
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"{""recipe_id"":1,""title"":""Chive Butter Radishes""...",None,,Classify the recipe into one of these types (M...
1,"{""recipe_id"":2,""title"":""Sweet Potato Breakfast...",None,,Classify the recipe into one of these types (M...
2,"{""recipe_id"":3,""title"":""Spicy Black Bean Nacho...",None,,Classify the recipe into one of these types (M...
3,"{""recipe_id"":4,""title"":""Balsamic Pork Chops"",""...",None,,Classify the recipe into one of these types (M...
4,"{""recipe_id"":1,""title"":""Chicken Noodle Soup"",""...",None,,Classify the recipe into one of these types (M...
5,"{""recipe_id"":6,""title"":""Pinto Beans and Tomati...",None,,Classify the recipe into one of these types (M...
6,"{""recipe_id"":7,""title"":""Turkey Soup with Homem...",None,,Classify the recipe into one of these types (M...
7,"{""recipe_id"":8,""title"":""Ham & Potato Soup"",""ty...",None,,Classify the recipe into one of these types (M...
8,"{""recipe_id"":9,""title"":""Spanish Chickpeas"",""ty...",None,,Classify the recipe into one of these types (M...
9,"{""recipe_id"":10,""title"":""Thai Veggie Soup"",""ty...",None,,Classify the recipe into one of these types (M...


In [ ]:
# there appear to be lots of soups that are being classified as main dishes. We will try listing main dish last so it recieves less attention and is only used if there is no better cateogry
%%bigquery
declare prompt_query STRING default "Classify the recipe into one of these types (Soup, Salad, Appetizer, Side Dish, Dessert, Drink, Breakfast, Main Dish) if it does not aready have a type. Return the output as json, include the recipe_id, title, and type";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "type", type))) as prompt
    from magazine_recipes_stg.Recipes
    order by recipe_id
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);


Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"{""recipe_id"":1,""title"":""Chive Butter Radishes""...",None,,Classify the recipe into one of these types (S...
1,"{""recipe_id"":2,""title"":""Sweet Potato Breakfast...",None,,Classify the recipe into one of these types (S...
2,"{""recipe_id"":3,""title"":""Spicy Black Bean Nacho...",None,,Classify the recipe into one of these types (S...
3,"{""recipe_id"":4,""title"":""Balsamic Pork Chops"",""...",None,,Classify the recipe into one of these types (S...
4,"{""recipe_id"":1,""title"":""Creamy Tomato Soup"",""t...",None,,Classify the recipe into one of these types (S...
5,"{""recipe_id"":6,""title"":""Pinto Beans and Tomati...",None,,Classify the recipe into one of these types (S...
6,"{""recipe_id"":7,""title"":""Turkey Soup with Homem...",None,,Classify the recipe into one of these types (S...
7,"{""recipe_id"":8,""title"":""Ham & Potato Soup"",""ty...",None,,Classify the recipe into one of these types (S...
8,"{""recipe_id"":9,""title"":""Spanish Chickpeas"",""ty...",None,,Classify the recipe into one of these types (S...
9,"{""recipe_id"":10,""title"":""Thai Veggie Soup"",""ty...",None,,Classify the recipe into one of these types (S...


In [ ]:
%%bigquery
declare prompt_query STRING default "Classify the recipe into one of these types (Soup, Salad, Appetizer, Side Dish, Dessert, Drink, Breakfast, Main Dish) if it does not aready have a type. Return the output as json, include the recipe_id, title, and type";
create or replace table magazine_recipes_stg.type_predictions_raw_10 as
  select *
  from ML.generate_text(
      model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "type", type))) as prompt
    from magazine_recipes_stg.Recipes
    order by recipe_id
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select json_value(ml_generate_text_llm_result, '$.recipe_id') as recipe_id,
  json_value(ml_generate_text_llm_result, '$.type') as type
from magazine_recipes_stg_ai.type_predictions_raw_10

Query is running:   0%|          |

Downloading:   0%|          |

,recipe_id,type
0,2,Breakfast
1,1,Soup
2,1,Appetizer
3,4,Main Dish
4,8,Soup
5,6,Main Dish
6,9,Main Dish
7,3,Appetizer
8,7,Main Dish
9,10,Soup


In [ ]:
%%bigquery
update magazine_recipes_stg.Recipes set type =
  (select json_value(ml_generate_text_llm_result, '$.type')
   from  magazine_recipes_stg_ai.type_predictions_raw_10
   where recipe_id = cast(json_value(ml_generate_text_llm_result, '$.recipe_id') as INT64))
where type is null

Executing query with job ID: 90c6a310-d141-4217-a00a-7816beeb6d51
Query executing: 0.56s


ERROR:
 400 Scalar subquery produced more than one element

Location: US
Job ID: 90c6a310-d141-4217-a00a-7816beeb6d51



In [ ]:
# The model got confused for records that already had a type, which resulted in non-unique recipe_id. We will try to clarify the prompt and see if recipe_id will be unique
%%bigquery
declare prompt_query STRING default "Classify the recipe into one of these types (Soup, Salad, Appetizer, Side Dish, Dessert, Drink, Breakfast, Main Dish) if it does not aready have a type. If it does have a type, assign the type listed. Return the output as json, include the recipe_id, title, and type";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "type", type))) as prompt
    from magazine_recipes_stg.Recipes
    order by recipe_id
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);


Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"{""recipe_id"":1,""title"":""Chive Butter Radishes""...",None,,Classify the recipe into one of these types (S...
1,"{""recipe_id"":2,""title"":""Sweet Potato Breakfast...",None,,Classify the recipe into one of these types (S...
2,"{""recipe_id"":3,""title"":""Spicy Black Bean Nacho...",None,,Classify the recipe into one of these types (S...
3,"{""recipe_id"":4,""title"":""Balsamic Pork Chops"",""...",None,,Classify the recipe into one of these types (S...
4,"{""recipe_id"":1,""title"":""Creamy Tomato Soup"",""t...",None,,Classify the recipe into one of these types (S...
5,"{""recipe_id"":6,""title"":""Pinto Beans and Tomati...",None,,Classify the recipe into one of these types (S...
6,"{""recipe_id"":7,""title"":""Turkey Soup with Homem...",None,,Classify the recipe into one of these types (S...
7,"{""recipe_id"":8,""title"":""Ham & Potato Soup"",""ty...",None,,Classify the recipe into one of these types (S...
8,"{""recipe_id"":9,""title"":""Spanish Chickpeas"",""ty...",None,,Classify the recipe into one of these types (S...
9,"{""recipe_id"":10,""title"":""Thai Veggie Soup"",""ty...",None,,Classify the recipe into one of these types (S...


In [ ]:
# This clarification did not work, let us try omitting the information about existing types
%%bigquery
declare prompt_query STRING default "Classify the recipe into one of these types (Soup, Salad, Appetizer, Side Dish, Dessert, Drink, Breakfast, Main Dish) Return the output as json, include the recipe_id, title, and type";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "type", type))) as prompt
    from magazine_recipes_stg.Recipes
    order by recipe_id
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);


Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"{""recipe_id"":1,""title"":""Chive Butter Radishes""...",None,,Classify the recipe into one of these types (S...
1,"{""recipe_id"":2,""title"":""Sweet Potato Breakfast...",None,,Classify the recipe into one of these types (S...
2,"{""recipe_id"":3,""title"":""Spicy Black Bean Nacho...",None,,Classify the recipe into one of these types (S...
3,"{""recipe_id"":4,""title"":""Balsamic Pork Chops"",""...",None,,Classify the recipe into one of these types (S...
4,"{""recipe_id"":5,""title"":""Chocolate Raspberry To...",None,,Classify the recipe into one of these types (S...
5,"{""recipe_id"":6,""title"":""Pinto Beans and Tomati...",None,,Classify the recipe into one of these types (S...
6,"{""recipe_id"":7,""title"":""Turkey Soup with Homem...",None,,Classify the recipe into one of these types (S...
7,"{""recipe_id"":8,""title"":""Ham & Potato Soup"",""ty...",None,,Classify the recipe into one of these types (S...
8,"{""recipe_id"":9,""title"":""Spanish Chickpeas"",""ty...",None,,Classify the recipe into one of these types (S...
9,"{""recipe_id"":10,""title"":""Thai Veggie Soup"",""ty...",None,,Classify the recipe into one of these types (S...


In [ ]:
# We have unique recipe_id now, but breakfast tacos are now a main dish. Let us give more explicit instructions about main dish and see if it helps
%%bigquery
declare prompt_query STRING default "Classify the recipe into one of these types (Soup, Salad, Appetizer, Side Dish, Dessert, Drink, Breakfast, Main Dish). Only assign main dish if it does not fit into one of the more precise types. Return the output as json, include the recipe_id, title, and type";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "type", type))) as prompt
    from magazine_recipes_stg.Recipes
    order by recipe_id
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);


Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"{""recipe_id"":1,""title"":""Chive Butter Radishes""...",None,,Classify the recipe into one of these types (S...
1,"{""recipe_id"":2,""title"":""Sweet Potato Breakfast...",None,,Classify the recipe into one of these types (S...
2,"{""recipe_id"":3,""title"":""Spicy Black Bean Nacho...",None,,Classify the recipe into one of these types (S...
3,"{""recipe_id"":4,""title"":""Balsamic Pork Chops"",""...",None,,Classify the recipe into one of these types (S...
4,"{""recipe_id"":5,""title"":""Chocolate Raspberry To...",None,,Classify the recipe into one of these types (S...
5,"{""recipe_id"":6,""title"":""Pinto Beans and Tomati...",None,,Classify the recipe into one of these types (S...
6,"{""recipe_id"":7,""title"":""Turkey Soup with Homem...",None,,Classify the recipe into one of these types (S...
7,"{""recipe_id"":8,""title"":""Ham & Potato Soup"",""ty...",None,,Classify the recipe into one of these types (S...
8,"{""recipe_id"":9,""title"":""Spanish Chickpeas"",""ty...",None,,Classify the recipe into one of these types (S...
9,"{""recipe_id"":10,""title"":""Thai Veggie Soup"",""ty...",None,,Classify the recipe into one of these types (S...


In [ ]:
# Breakfast tacos were still classified as main dish, let us try another clarification
%%bigquery
declare prompt_query STRING default "Classify the recipe into one of these types (Soup, Salad, Dessert, Breakfast, Appetizer, Drink, Side Dish, Main Dish). Evaluate if it can be classified as one of the types in the order they are listed. If the type is in the title, assign that as the type. Return the output as json, include the recipe_id, title, and type";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "type", type))) as prompt
    from magazine_recipes_stg.Recipes
    order by recipe_id
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);


Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"{""recipe_id"":1,""title"":""Chive Butter Radishes""...",None,,Classify the recipe into one of these types (S...
1,"{""recipe_id"":2,""title"":""Sweet Potato Breakfast...",None,,Classify the recipe into one of these types (S...
2,"{""recipe_id"":3,""title"":""Spicy Black Bean Nacho...",None,,Classify the recipe into one of these types (S...
3,"{""recipe_id"":4,""title"":""Balsamic Pork Chops"",""...",None,,Classify the recipe into one of these types (S...
4,"{""recipe_id"":5,""title"":""Chocolate Raspberry To...",None,,Classify the recipe into one of these types (S...
5,"{""recipe_id"":6,""title"":""Pinto Beans and Tomati...",None,,Classify the recipe into one of these types (S...
6,"{""recipe_id"":7,""title"":""Turkey Soup with Homem...",None,,Classify the recipe into one of these types (S...
7,"{""recipe_id"":8,""title"":""Ham & Potato Soup"",""ty...",None,,Classify the recipe into one of these types (S...
8,"{""recipe_id"":9,""title"":""Spanish Chickpeas"",""ty...",None,,Classify the recipe into one of these types (S...
9,"{""recipe_id"":10,""title"":""Thai Veggie Soup"",""ty...",None,,Classify the recipe into one of these types (S...


In [ ]:
# We are happy with the classifications and can update the table in BQ
%%bigquery
declare prompt_query STRING default "Classify the recipe into one of these types (Soup, Salad, Dessert, Breakfast, Appetizer, Drink, Side Dish, Main Dish). Evaluate if it can be classified as one of the types in the order they are listed. If the type is in the title, assign that as the type. Return the output as json, include the recipe_id, title, and type";
create or replace table magazine_recipes_stg_ai.type_predictions_raw_10 as
  select *
  from ML.generate_text(
      model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "type", type))) as prompt
    from magazine_recipes_stg.Recipes
    order by recipe_id
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select json_value(ml_generate_text_llm_result, '$.recipe_id') as recipe_id,
  json_value(ml_generate_text_llm_result, '$.title') as title,
  json_value(ml_generate_text_llm_result, '$.type') as type
from magazine_recipes_stg_ai.type_predictions_raw_10
order by recipe_id

Query is running:   0%|          |

Downloading:   0%|          |

,recipe_id,title,type
0,1,Chive Butter Radishes,Appetizer
1,10,Thai Veggie Soup,Soup
2,2,Sweet Potato Breakfast Burritos,Breakfast
3,3,Spicy Black Bean Nachos,Main Dish
4,4,Balsamic Pork Chops,Main Dish
5,5,Chocolate Raspberry Torte,Dessert
6,6,Pinto Beans and Tomatillo Cilantro Lime Rice,Main Dish
7,7,Turkey Soup with Homemade Noodles,Main Dish
8,8,Ham & Potato Soup,Soup
9,9,Spanish Chickpeas,Main Dish


In [ ]:
# Update the recipes table
%%bigquery
update magazine_recipes_stg.Recipes set type =
  (select json_value(ml_generate_text_llm_result, '$.type')
   from magazine_recipes_stg_ai.type_predictions_raw_10
   where recipe_id = cast(json_value(ml_generate_text_llm_result, '$.recipe_id') as int64))
where 1 = 1

Query is running:   0%|          |

""


#### Inspect the output

In [ ]:
# check the output- subset by recipe_id since many other records already have non-AI generated types
%%bigquery
select *
from magazine_recipes_stg.Recipes
where type is not null and recipe_id < 10

Query is running:   0%|          |

Downloading:   0%|          |

,recipe_id,title,subtitle,servings,yield_unit,prep_min,cook_min,stnd_min,source,intro,...,rating,ease_of_prep,note,type,page,slowcooker,link,last_made,data_source,load_time
0,4,Balsamic Pork Chops,None,<NA>,None,<NA>,<NA>,<NA>,None,None,...,1,None,None,Main Dish,<NA>,None,None,2019-02-14,airtable,2024-02-02 21:09:57.475069+00:00
1,3,Spicy Black Bean Nachos,None,<NA>,None,<NA>,<NA>,<NA>,None,None,...,2,None,None,Main Dish,<NA>,None,https://www.mexicanplease.com/spicy-black-bean...,2018-09-17,airtable,2024-02-02 21:09:57.475069+00:00
2,2,Sweet Potato Breakfast Burritos,None,<NA>,None,<NA>,<NA>,<NA>,None,None,...,4,None,None,Breakfast,<NA>,None,https://www.ambitiouskitchen.com/healthy-sweet...,2018-11-01,airtable,2024-02-02 21:09:57.475069+00:00
3,1,Chive Butter Radishes,None,<NA>,None,<NA>,<NA>,<NA>,None,None,...,4,None,None,Appetizer,<NA>,None,None,2018-06-16,airtable,2024-02-02 21:09:57.475069+00:00
4,9,Spanish Chickpeas,None,<NA>,None,45,<NA>,<NA>,None,None,...,5,Average,Don't forget to make Naan dough ahead of time,Main Dish,<NA>,None,https://www.prevention.com/food/10-healthy-chi...,2019-01-21,airtable,2024-02-02 21:09:57.475069+00:00
5,5,Chocolate Raspberry Torte,None,<NA>,None,60,<NA>,<NA>,None,None,...,5,Hard,So good,Dessert,<NA>,None,None,2016-10-11,airtable,2024-02-02 21:09:57.475069+00:00
6,8,Ham & Potato Soup,None,<NA>,None,60,<NA>,<NA>,None,None,...,4,Average,Super yummy,Soup,<NA>,None,http://allrecipes.com/recipe/56927/delicious-h...,2018-07-15,airtable,2024-02-02 21:09:57.475069+00:00
7,6,Pinto Beans and Tomatillo Cilantro Lime Rice,None,<NA>,None,60,<NA>,<NA>,None,None,...,5,Hard,Amazing flavor,Main Dish,<NA>,None,https://www.thecuriouschickpea.com/mexican-pin...,2019-02-17,airtable,2024-02-02 21:09:57.475069+00:00
8,7,Turkey Soup with Homemade Noodles,None,<NA>,None,120,<NA>,<NA>,None,None,...,4,Average,None,Main Dish,<NA>,None,None,2016-11-17,airtable,2024-02-02 21:09:57.475069+00:00


#### Apply at larger scale

In [ ]:
# We do not have to reformat the json, so no need to name the table raw since it will be the final one
%%bigquery
declare prompt_query STRING default "Classify the recipe into one of these types (Soup, Salad, Dessert, Breakfast, Appetizer, Drink, Side Dish, Main Dish). Evaluate if it can be classified as one of the types in the order they are listed. If the type is in the title, assign that as the type. Return the output as json, include the recipe_id, title, and type";
create or replace table magazine_recipes_stg_ai.type_predictions as
  select *
  from ML.generate_text(
      model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "type", type))) as prompt
    from magazine_recipes_stg.Recipes
    order by recipe_id
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

""


Query only took 5 minutes to run since there were under 1000 recipes in the table

In [ ]:
%%bigquery
update magazine_recipes_stg.Recipes r
  set type = json_value(p.ml_generate_text_llm_result, '$.type')
  from magazine_recipes_stg_ai.type_predictions p
  where r.recipe_id = cast(json_value(p.ml_generate_text_llm_result, '$.recipe_id') as int64);

Executing query with job ID: 676e4d64-a069-494d-859f-781f0b1d4183
Query executing: 0.22s


ERROR:
 400 Syntax error: Expected end of input but got keyword WHERE at [5:1]

Location: US
Job ID: 676e4d64-a069-494d-859f-781f0b1d4183



In [ ]:
# check to see if we are missing any types
%%bigquery
select count(*) as missing_types
from magazine_recipes_stg.Recipes
where type is null

Query is running:   0%|          |

Downloading:   0%|          |

,type_count
0,307


In [ ]:
%%bigquery
select * from magazine_recipes_stg.Recipes
where type is null
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,recipe_id,title,subtitle,servings,yield_unit,prep_min,cook_min,stnd_min,source,intro,...,rating,ease_of_prep,note,type,page,slowcooker,link,last_made,data_source,load_time
0,43,Potato Apple Roast,None,<NA>,None,45,<NA>,<NA>,Pinterest,None,...,4,Average,None,None,<NA>,None,https://www.healthyseasonalrecipes.com/roasted...,2018-07-21,airtable,2024-02-02 21:09:57.475069+00:00
1,142,Dill Cucumber Salmon,None,<NA>,None,30,<NA>,<NA>,Sheet Pan Suppers,None,...,4,Average,None,None,<NA>,None,None,2018-11-02,airtable,2024-02-02 21:09:57.475069+00:00
2,117,Cloverleaf Rolls,None,<NA>,None,30,<NA>,<NA>,Taste From Home,None,...,3,Fairly Easy,None,None,<NA>,None,None,NaT,airtable,2024-02-02 21:09:57.475069+00:00
3,45,Malteese Gilatti,None,<NA>,None,120,<NA>,<NA>,Pinterest,None,...,2,Average,Hard to get them toasty,None,<NA>,None,http://www.ilovefood.com.mt/recipes/galletti-r...,2017-12-17,airtable,2024-02-02 21:09:57.475069+00:00
4,61,Buckwheat Carrot and Onion,None,<NA>,None,40,<NA>,<NA>,Pinterest,None,...,4,Fairly Easy,None,None,<NA>,None,http://www.pinchofcinnamon.com/2011/12/cooking...,2018-01-10,airtable,2024-02-02 21:09:57.475069+00:00


In [ ]:
# Add direction telling the model to assign a type to every recipe
%%bigquery
declare prompt_query STRING default "Classify the recipe into one of these types (Soup, Salad, Dessert, Breakfast, Appetizer, Drink, Side Dish, Main Dish). Do not return a null type. Evaluate if it can be classified as one of the types in the order they are listed. If the type is in the title, assign that as the type. Return the output as json, include the recipe_id, title, and type";
create or replace table magazine_recipes_stg_ai.type_predictions as
  select *
  from ML.generate_text(
      model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "type", type))) as prompt
    from magazine_recipes_stg.Recipes
    order by recipe_id
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery
update magazine_recipes_stg.Recipes r
  set type = json_value(p.ml_generate_text_llm_result, '$.type')
  from magazine_recipes_stg_ai.type_predictions p
  where r.recipe_id = cast(json_value(p.ml_generate_text_llm_result, '$.recipe_id') as int64);

Query is running:   0%|          |

""


In [ ]:
# check to see if we are missing any types
%%bigquery
select count(*) as missing_types
from magazine_recipes_stg.Recipes
where type is null

Query is running:   0%|          |

Downloading:   0%|          |

,missing_types
0,0


In [ ]:
# note that the model added a few categories such as Miscellaneous and dressings
%%bigquery
select type, count(*) as type_count
from magazine_recipes_stg.Recipes
group by type
order by type_count desc

Query is running:   0%|          |

Downloading:   0%|          |

,type,type_count
0,Main Dish,367
1,Miscellaneous,187
2,Salad,130
3,Dessert,106
4,Side Dish,74
5,Soup,46
6,Drink,40
7,Appetizer,34
8,Breakfast,34
9,Dressing,4


In [ ]:
# since there is only one recipe that is a sauce and 4 dressings, we will combine them into one category
%%bigquery
update magazine_recipes_stg.Recipes
  set type = 'Dressing/Sauce'
where type in ('Dressing', 'Sauce')

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select type, count(*) as type_count
from magazine_recipes_stg.Recipes
group by type
order by type_count desc

Query is running:   0%|          |

Downloading:   0%|          |

,type,type_count
0,Main Dish,367
1,Miscellaneous,187
2,Salad,130
3,Dessert,106
4,Side Dish,74
5,Soup,46
6,Drink,40
7,Appetizer,34
8,Breakfast,34
9,Dressing/Sauce,5


In [ ]:
# we will not update the datasource because the type will be present in the pages table and not in the recipes table

## Clean ingredients table
Initally we planned to use AI to give categories to airtable ingredients not found in bird, but we realized that the ingredients were likely already in bird but under a slightly different name, we will use the next section to clean up the ingredients table before continuing on with project 9

In [ ]:
%%bigquery
select * from magazine_recipes_stg.Ingredients
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,ingredient_id,category,name,plural,data_source,load_time
0,4648,None,carrots,None,Airtable,2024-02-02 21:09:57.475069+00:00
1,4666,None,adobo chile,None,Airtable,2024-02-02 21:09:57.475069+00:00
2,4661,None,beets,None,Airtable,2024-02-02 21:09:57.475069+00:00
3,4655,None,vegetarian,None,Airtable,2024-02-02 21:09:57.475069+00:00
4,4660,None,kale,None,Airtable,2024-02-02 21:09:57.475069+00:00
5,4656,None,potatoes,None,Airtable,2024-02-02 21:09:57.475069+00:00
6,4658,None,chocolate chips,None,Airtable,2024-02-02 21:09:57.475069+00:00
7,4646,None,bell pepper,None,Airtable,2024-02-02 21:09:57.475069+00:00
8,4663,None,parsnips,None,Airtable,2024-02-02 21:09:57.475069+00:00
9,4653,None,gnocchi,None,Airtable,2024-02-02 21:09:57.475069+00:00


In [ ]:
# oh no! It appears we forgot to account for plurals when we merged the airtable and bird ingreidient tables :( lets remedy this!
%%bigquery
select * from magazine_recipes_stg.Ingredients
where name = 'carrot'

Query is running:   0%|          |

Downloading:   0%|          |

,ingredient_id,category,name,plural,data_source,load_time
0,2260,fresh vegetables,carrot,s,bird,2024-01-26 23:50:31.637778+00:00


In [ ]:
%%bigquery
select *
from magazine_recipes_stg.Ingredients
where ENDS_WITH(name, 's') = True and data_source = 'Airtable'

Query is running:   0%|          |

Downloading:   0%|          |

,ingredient_id,category,name,plural,data_source,load_time
0,4648,None,carrots,None,Airtable,2024-02-02 21:09:57.475069+00:00
1,4661,None,beets,None,Airtable,2024-02-02 21:09:57.475069+00:00
2,4656,None,potatoes,None,Airtable,2024-02-02 21:09:57.475069+00:00
3,4658,None,chocolate chips,None,Airtable,2024-02-02 21:09:57.475069+00:00
4,4663,None,parsnips,None,Airtable,2024-02-02 21:09:57.475069+00:00
5,4662,None,tomatoes,None,Airtable,2024-02-02 21:09:57.475069+00:00
6,4664,None,eggs,None,Airtable,2024-02-02 21:09:57.475069+00:00
7,4657,None,breadcrumbs,None,Airtable,2024-02-02 21:09:57.475069+00:00
8,4649,None,brussels sprouts,None,Airtable,2024-02-02 21:09:57.475069+00:00
9,4651,None,shallots,None,Airtable,2024-02-02 21:09:57.475069+00:00


In [ ]:
%%bigquery
create or replace table magazine_recipes_stg.Ingredients_duplicates as
select * from
((select ingredient_id as wrong_id, LEFT(name, LENGTH(name) - 1) as at_name
from magazine_recipes_stg.Ingredients
where ENDS_WITH(name, 's') = True
and data_source = 'Airtable'
and LEFT(name, LENGTH(name) - 1) in (select name from magazine_recipes_stg.Ingredients where data_source = 'bird'))
UNION ALL
(select ingredient_id as wrong_id, LEFT(name, LENGTH(name) - 2) at_name
from magazine_recipes_stg.Ingredients
where ENDS_WITH(name, 's') = True
and data_source = 'Airtable'
and LEFT(name, LENGTH(name) - 2) in (select name from magazine_recipes_stg.Ingredients where data_source = 'bird'))) a
left join (select name, ingredient_id from magazine_recipes_stg.Ingredients) b on a.at_name = b.name

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select *
from magazine_recipes_stg.Ingredients_duplicates

Query is running:   0%|          |

Downloading:   0%|          |

,wrong_id,at_name,name,ingredient_id
0,4656,potato,potato,2286
1,4662,tomato,tomato,3480
2,4661,beet,beet,199
3,4651,shallot,shallot,3093
4,4648,carrot,carrot,2260
5,4663,parsnip,parsnip,2534
6,4664,egg,egg,1171
7,4658,chocolate chip,chocolate chip,707


In [ ]:
# now we have this table of the old and new ingredient ids for the duplicate ingredients that we can use to update the junction table before we delete the incorrect records from ingredients
%%bigquery
update magazine_recipes_stg.Quantity
set ingredient_id = (select ingredient_id from magazine_recipes_stg.Ingredients_duplicates where wrong_id = ingredient_id)
where ingredient_id in (select wrong_id from magazine_recipes_stg.Ingredients_duplicates)

Query is running:   0%|          |

""


In [ ]:
# check if the updates worked in junction table
%%bigquery
select count(*) as wrong_ids from magazine_recipes_stg.Quantity
where ingredient_id in (select wrong_id from magazine_recipes_stg.Ingredients_duplicates)

Query is running:   0%|          |

Downloading:   0%|          |

,wrong_ids
0,0


In [ ]:
# delete the duplicate records from ingredients
%%bigquery
delete from magazine_recipes_stg.Ingredients
where ingredient_id in (select wrong_id from magazine_recipes_stg.Ingredients_duplicates)

Query is running:   0%|          |

""


In [ ]:
# update the datasource for those ingreidents
%%bigquery
update magazine_recipes_stg.Ingredients
set data_source = 'bird-airtable'
where ingredient_id in (select wrong_id from magazine_recipes_stg.Ingredients_duplicates)

Query is running:   0%|          |

""


In [ ]:
# it appears that these are likely in bird but labeled slighly differenly, since there are only 15, we will go in by hand and edit them
%%bigquery
select * from magazine_recipes_stg.Ingredients
where data_source = 'Airtable'

Query is running:   0%|          |

Downloading:   0%|          |

,ingredient_id,category,name,plural,data_source,load_time
0,4668,None,peanut,None,Airtable,2024-02-02 21:09:57.475069+00:00
1,4657,None,breadcrumbs,None,Airtable,2024-02-02 21:09:57.475069+00:00
2,4665,None,wine,None,Airtable,2024-02-02 21:09:57.475069+00:00
3,4649,None,brussels sprouts,None,Airtable,2024-02-02 21:09:57.475069+00:00
4,4654,None,jalopeno pepper,None,Airtable,2024-02-02 21:09:57.475069+00:00
5,4647,None,tilapia,None,Airtable,2024-02-02 21:09:57.475069+00:00
6,4666,None,adobo chile,None,Airtable,2024-02-02 21:09:57.475069+00:00
7,4659,None,chia,None,Airtable,2024-02-02 21:09:57.475069+00:00
8,4653,None,gnocchi,None,Airtable,2024-02-02 21:09:57.475069+00:00
9,4667,None,tuna,None,Airtable,2024-02-02 21:09:57.475069+00:00


In [ ]:
%%bigquery
update magazine_recipes_stg.Quantity
set ingredient_id = 1025
where ingredient_id = 4650


Query is running:   0%|          |

""


In [ ]:
%%bigquery
update magazine_recipes_stg.Ingredients
set data_source = 'bird_airtable'
where ingredient_id = 1025

Query is running:   0%|          |

""


In [ ]:
%%bigquery
delete from magazine_recipes_stg.Ingredients
where ingredient_id = 4650

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select *
from magazine_recipes_stg.Ingredients
where CONTAINS_SUBSTR(name, 'dijon') = True
order by ingredient_id

Query is running:   0%|          |

Downloading:   0%|          |

,ingredient_id,category,name,plural,data_source,load_time
0,763,condiments/sauces,coarse Dijon-style mustard,None,bird,2024-01-26 23:50:31.637778+00:00
1,896,condiments/sauces,country-style Dijon mustard,None,bird,2024-01-26 23:50:31.637778+00:00
2,1025,condiments/sauces,Dijon mustard,None,bird,2024-01-26 23:50:31.637778+00:00
3,3778,salad dressings,Wish-Bone Healthy Sensation! Honey Dijon,None,bird,2024-01-26 23:50:31.637778+00:00
4,4650,None,dijon mustard,None,Airtable,2024-02-02 21:09:57.475069+00:00


In [ ]:
# make sure we don't have any orphans in quantity, we don't yay!
%%bigquery
select count(*) as orphans from magazine_recipes_stg.Quantity
where ingredient_id not in (select ingredient_id from magazine_recipes_stg.Ingredients)

Query is running:   0%|          |

Downloading:   0%|          |

,orphans
0,0


# Change 2: Create User names based on First Name Last Name, State, and fun persona

In [ ]:
# assign usernames to the users table
%%bigquery
declare prompt_query STRING default "Assign a user name for a recipe sharing platform based on user data and other fun terms commonly found in usernames or related to cooking. Only include numbers and letters in usernames. Get creative and have fun with it. Return the output as json, include the numeric user_id, first name, and last name and creative user name";
  select *
  from ML.generate_text(
      model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("user_id", user_id, "first_name", f_name, "last_name", l_name, "age", age))) as prompt
    from magazine_recipes_stg.Users
    order by user_id
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"{""user_id"":1,""first_name"":""Norma"",""last_name"":...",None,,Assign a user name for a recipe sharing platfo...
1,"```json\n{\n ""user_id"": 2,\n ""first_name"": ""...",None,,Assign a user name for a recipe sharing platfo...
2,"{""user_id"":3,""first_name"":""Beyonce"",""last_name...",None,,Assign a user name for a recipe sharing platfo...
3,"```json\n{\n ""user_id"": 4,\n ""first_name"": ""...",None,,Assign a user name for a recipe sharing platfo...
4,"```json\n{\n ""user_id"": 5,\n ""first_name"": ""...",None,,Assign a user name for a recipe sharing platfo...
5,"```json\n{\n ""user_id"": 6,\n ""first_name"": ""...",None,,Assign a user name for a recipe sharing platfo...
6,"{""user_id"":7,""first_name"":""Samantha"",""last_nam...",None,,Assign a user name for a recipe sharing platfo...
7,"```json\n{\n ""user_id"": 8,\n ""first_name"": ""...",None,,Assign a user name for a recipe sharing platfo...
8,"{""user_id"":9,""first_name"":""Lisa"",""last_name"":""...",None,,Assign a user name for a recipe sharing platfo...
9,"```json\n{\n ""user_id"": 10,\n ""first_name"": ...",None,,Assign a user name for a recipe sharing platfo...


In [ ]:
%%bigquery
alter table magazine_recipes_stg.Users add column username String;

Query is running:   0%|          |

""


In [ ]:
# apply at larger scale
%%bigquery
declare prompt_query STRING default "Assign a user name for a recipe sharing platform based on user data and other fun terms commonly found in usernames or related to cooking. Only include numbers and letters in usernames. Get creative and have fun with it. Return the output as json, include the numeric user_id, first name, and last name and creative user name";
  create or replace table magazine_recipes_stg_ai.usernames as
  select *
  from ML.generate_text(
      model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("user_id", user_id, "first_name", f_name, "last_name", l_name, "age", age))) as prompt
    from magazine_recipes_stg.Users
    order by user_id
  ),
  struct(TRUE as flatten_json_output)
);


Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from magazine_recipes_stg_ai.usernames
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"```json\n{\n ""user_id"": 13,\n ""first_name"": ...",None,,Assign a user name for a recipe sharing platfo...
1,"```json\n{\n ""user_id"": 25,\n ""first_name"": ...",None,,Assign a user name for a recipe sharing platfo...
2,"{""user_id"":54,""first_name"":""Janice"",""last_name...",None,,Assign a user name for a recipe sharing platfo...
3,"```json\n{\n ""user_id"": 63,\n ""first_name"": ...",None,,Assign a user name for a recipe sharing platfo...
4,"```json\n{\n ""user_id"": 75,\n ""first_name"": ...",None,,Assign a user name for a recipe sharing platfo...
5,"```json\n{\n ""user_id"": 28,\n ""first_name"": ...",None,,Assign a user name for a recipe sharing platfo...
6,"```json\n{\n ""user_id"": 45,\n ""first_name"": ...",None,,Assign a user name for a recipe sharing platfo...
7,"{""age"":47,""first_name"":""Sarah"",""last_name"":""La...",None,,Assign a user name for a recipe sharing platfo...
8,"{""age"":36,""first_name"":""Eric"",""last_name"":""Pri...",None,,Assign a user name for a recipe sharing platfo...
9,"```json\n{\n ""user_id"": 10,\n ""first_name"": ...",None,,Assign a user name for a recipe sharing platfo...


In [ ]:
%%bigquery
update magazine_recipes_stg.Users set username =
  (select json_value(ml_generate_text_llm_result, '$.user_name')
   from magazine_recipes_stg_ai.usernames
   where user_id = CAST(json_value(ml_generate_text_llm_result, '$.user_id') as INT64))
where 1=1;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select count(*)
from magazine_recipes_stg.Users
where username is NULL

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,64


In [ ]:
# some json fields are named username rather than user_name
%%bigquery
update magazine_recipes_stg.Users set username =
  (select json_value(ml_generate_text_llm_result, '$.username')
   from magazine_recipes_stg_ai.usernames
   where user_id = CAST(json_value(ml_generate_text_llm_result, '$.user_id') as INT64))
where username is null;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select count(*)
from magazine_recipes_stg.Users
where username is NULL

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,47


In [ ]:
# I will just re-ask ai and specify what the new field should be called so that there are no nulls
%%bigquery
declare prompt_query STRING default "Assign a user name for a recipe sharing platform based on user data and other fun terms commonly found in usernames or related to cooking. Only include numbers and letters in usernames. Get creative and have fun with it. Return the output as json, include the numeric user_id, first name, and last name and creative user name (named the json field user_name)";
  create or replace table magazine_recipes_stg_ai.usernames as
  select *
  from ML.generate_text(
      model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("user_id", user_id, "first_name", f_name, "last_name", l_name, "age", age))) as prompt
    from magazine_recipes_stg.Users
    order by user_id
  ),
  struct(TRUE as flatten_json_output)
);


Query is running:   0%|          |

""


In [ ]:
%%bigquery
update magazine_recipes_stg.Users set username =
  (select json_value(ml_generate_text_llm_result, '$.user_name')
   from magazine_recipes_stg_ai.usernames
   where user_id = CAST(json_value(ml_generate_text_llm_result, '$.user_id') as INT64))
where 1=1;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select count(*)
from magazine_recipes_stg.Users
where username is NULL

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,0


In [ ]:
# update the datasource for users
%%bigquery
create or replace table magazine_recipes_stg.Users as
select user_id, username, f_name, l_name, age, phone, state, 'faker-ai' as data_source, load_time
from magazine_recipes_stg.Users

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select *
from magazine_recipes_stg.Publications
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,publication_id,recipe_id,magazine_id,journalist_id,date,volume,issue,publication_type,data_source,load_time
0,0,1471,7,38,NaT,<NA>,<NA>,None,None,2024-02-10 01:05:38.042312+00:00
1,1,1559,17,17,NaT,<NA>,<NA>,None,None,2024-02-10 01:05:38.042312+00:00
2,2,1509,14,25,NaT,<NA>,<NA>,None,None,2024-02-10 01:05:38.042312+00:00
3,3,1458,4,81,NaT,<NA>,<NA>,None,None,2024-02-10 01:05:38.042312+00:00
4,4,1567,17,29,NaT,<NA>,<NA>,None,None,2024-02-10 01:05:38.042312+00:00


In [ ]:
# since we are no longer doing magazines, we must change the publications table to posts
%%bigquery
create or replace table magazine_recipes_stg.Posts as
select p.publication_id as post_id, p.recipe_id, p.journalist_id as user_id, r.type as page, TIMESTAMP_SUB(p.load_time, INTERVAL CAST(FLOOR(RAND() * 2000) AS INT64) DAY) as timestamp_posted, 'ai' as datasource, p.load_time as load_time
from magazine_recipes_stg.Publications p
join magazine_recipes_stg.Recipes r
on p.recipe_id = r.recipe_id

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select *
from magazine_recipes_stg.Posts
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,post_id,recipe_id,user_id,page,timestamp_posted,datasource,load_time
0,1073,139,1,Soup,2020-05-17 01:05:38.042312+00:00,ai,2024-02-10 01:05:38.042312+00:00
1,641,1187,5,Soup,2021-10-04 01:05:38.042312+00:00,ai,2024-02-10 01:05:38.042312+00:00
2,1056,68,5,Soup,2021-11-20 01:05:38.042312+00:00,ai,2024-02-10 01:05:38.042312+00:00
3,1004,557,9,Soup,2022-12-02 01:05:38.042312+00:00,ai,2024-02-10 01:05:38.042312+00:00
4,969,10,16,Soup,2019-01-28 01:05:38.042312+00:00,ai,2024-02-10 01:05:38.042312+00:00


In [ ]:
# remove the type field from recipes because it is now in posts
%%bigquery
create or replace table magazine_recipes_stg.Recipes as
select * except(type)
from magazine_recipes_stg.Recipes

Query is running:   0%|          |

""


# Change 3:  Use AI to add a health rating to the nutrition table based on Nutritional value


In [ ]:
%%bigquery
declare prompt_query STRING default "Rate the nutritional value of the recipe from 1-5. 1 being unhealthy and 5 being healthy. Return output as json with recipe_id ";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", n.recipe_id, "title", title,
                  "carbo", carbo, "total_fat", total_fat, "cholestrl", cholestrl, "sodium", sodium, "calories",calories, "iron", iron, "vitamin_c", vitamin_c, "vitamin_a", vitamin_a))) as prompt
    from magazine_recipes_stg.Nutrition n
    inner join magazine_recipes_stg.Recipes r on r.recipe_id = n.recipe_id
    limit 20
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"```json\n{\n ""recipe_id"": 895,\n ""calories"":...",None,,Rate the nutritional value of the recipe from ...
1,"```json\n{""calories"":16.64,""carbo"":4.16,""chole...",None,,Rate the nutritional value of the recipe from ...
2,"```json\n{\n ""calories"": 15.75,\n ""carbo"": 2...",None,,Rate the nutritional value of the recipe from ...
3,"```json\n{\n ""recipe_id"": 1172,\n ""calories""...",None,,Rate the nutritional value of the recipe from ...
4,"```json\n{\n ""calories"": 139.13,\n ""carbo"": ...",None,,Rate the nutritional value of the recipe from ...
5,"```json\n{\n ""recipe_id"": 916,\n ""calories"":...",None,,Rate the nutritional value of the recipe from ...
6,"```json\n{\n ""recipe_id"": 1084,\n ""calories""...",None,,Rate the nutritional value of the recipe from ...
7,"```json\n{\n ""recipe_id"": 1346,\n ""title"": ""...",None,,Rate the nutritional value of the recipe from ...
8,"```json\n{\n ""recipe_id"": 586,\n ""rating"": 3...",None,,Rate the nutritional value of the recipe from ...
9,"```json\n{\n ""recipe_id"": 819,\n ""nutritiona...",None,,Rate the nutritional value of the recipe from ...


In [ ]:
# None of the recipes were being rated 1 or 5. THey were all mostly 3 or 3, so we narrow the ratings down further.
%%bigquery
declare prompt_query STRING default "Rate the nutritional value as: 1,2 or 3. 1 being unhealthy, 2 being neutral, and 3 being  healthy. Return recipe_id, title, rating and nutritional values. Rating must not be null";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", n.recipe_id, "title", title,
                  "carbo", carbo, "total_fat", total_fat, "cholestrl", cholestrl, "sodium", sodium, "calories",calories, "iron", iron, "vitamin_c", vitamin_c, "vitamin_a", vitamin_a))) as prompt
    from magazine_recipes_stg.Nutrition n
    inner join magazine_recipes_stg.Recipes r on r.recipe_id = n.recipe_id
    limit 20
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"{""recipe_id"":895,""title"":""-Soy Sauce-"",""rating...",None,,"Rate the nutritional value as: 1,2 or 3. 1 bei..."
1,"{""recipe_id"":900,""title"":""-White Granulated Su...",None,,"Rate the nutritional value as: 1,2 or 3. 1 bei..."
2,"{""recipe_id"":896,""title"":""-Teriyaki Sauce-"",""r...",None,,"Rate the nutritional value as: 1,2 or 3. 1 bei..."
3,"{""recipe_id"":1172,""title"":""Garlic-Soy Dipping ...",None,,"Rate the nutritional value as: 1,2 or 3. 1 bei..."
4,"{""recipe_id"":928,""title"":""-Tortilla Chips-"",""r...",None,,"Rate the nutritional value as: 1,2 or 3. 1 bei..."
5,"{""recipe_id"":916,""title"":""-Steak Sauce-"",""rati...",None,,"Rate the nutritional value as: 1,2 or 3. 1 bei..."
6,"{""recipe_id"":1084,""title"":""Sonoma Turkey Risot...",None,,"Rate the nutritional value as: 1,2 or 3. 1 bei..."
7,"{""recipe_id"":1346,""title"":""Grilled Whole Salmo...",None,,"Rate the nutritional value as: 1,2 or 3. 1 bei..."
8,"{""recipe_id"":586,""title"":""Lima Beans and Ham"",...",None,,"Rate the nutritional value as: 1,2 or 3. 1 bei..."
9,"{""recipe_id"":819,""title"":""-Millet-"",""rating"":2...",None,,"Rate the nutritional value as: 1,2 or 3. 1 bei..."


In [ ]:
# Use the averages as a baseline
%%bigquery
SELECT
  avg(cholestrl) as avg_cholestrl,
  avg(calories) as avg_calories,
  avg(sodium) as avg_sodium,
  avg(total_fat) as avg_total_fat,
  avg(carbo) as avg_carbo,
  avg(protien) as avg_protien,
  avg(iron) as avg_iron,
  avg(vitamin_c) as avg_vitamin_c,
  avg(vitamin_a) as avg_vitamin_a,
  avg(fiber) as avg_fiber
FROM magazine_recipes_stg.Nutrition

Query is running:   0%|          |

Downloading:   0%|          |

,avg_cholestrl,avg_calories,avg_sodium,avg_total_fat,avg_carbo,avg_protien,avg_iron,avg_vitamin_c,avg_vitamin_a,avg_fiber
0,36.166344,217.914465,341.498519,6.579032,26.963132,12.203189,2.033702,23.133554,1721.366503,1.124624


In [ ]:
# Prompt above was not descriptive enough, so we calculate exact numbers to compare the values to.
%%bigquery
declare prompt_query STRING default "Rate the nutritional value as: 1,2 or 3. 1 being unhealthy, 2 being neutral, and 3 being  healthy. cholestrol above 36 is unhealthy, sodium above 341 is unhealthy, total fat above 18 is unhealthy, carbo above 26 is unhealthy, protein below 12 is unhealthy, iron below 2 is unhealthy, vitamin_c below 23 is unhealthy, vitamin_a below 1721 is unhealthy, fiber below 1 is unhealthy. If less than 5 metrics are unhealthy, the rating is 3. If 5 metrics are unhealthy, the rating is 2. If more than 5 metrics are unhealthy, the rating is 1. Return recipe_id, title, rating, nutritional values. Rating must not be null";
create or replace table magazine_recipes_stg_ai.nutrition_ratings_10 as
  select *
  from ML.generate_text(
    model remote_models.gemini_pro,
    (
      select concat(prompt_query, to_json_string(json_object("recipe_id", n.recipe_id, "title", title,
                    "carbo", carbo, "total_fat", total_fat, "cholestrl", cholestrl, "sodium", sodium, "calories",calories, "iron", iron, "vitamin_c", vitamin_c, "vitamin_a", vitamin_a))) as prompt
      from magazine_recipes_stg.Nutrition n
      inner join magazine_recipes_stg.Recipes r on r.recipe_id = n.recipe_id
      limit 20
    ),
    struct(TRUE as flatten_json_output)
  );

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select ml_generate_text_llm_result, prompt
from magazine_recipes_stg_ai.nutrition_ratings_10

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,prompt
0,"{""calories"":8.28,""carbo"":2.07,""cholestrl"":0.0,...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
1,"{""calories"":462.78,""carbo"":27.39,""cholestrl"":1...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
2,"{""calories"":482.79,""carbo"":1.38,""cholestrl"":18...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
3,"{""calories"":372.87,""carbo"":40.37,""cholestrl"":8...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
4,"{""calories"":15.75,""carbo"":2.87,""cholestrl"":0.0...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
5,"{""calories"":535.29,""carbo"":75.82,""cholestrl"":4...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
6,"{""recipe_id"": 586, ""title"": ""Lima Beans and Ha...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
7,"{""recipe_id"": 1172, ""title"": ""Garlic-Soy Dippi...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
8,"{""recipe_id"":917,""title"":""-Lowfat Cottage Chee...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
9,"{""calories"":94.2,""carbo"":18.37,""cholestrl"":0.0...","Rate the nutritional value as: 1,2 or 3. 1 bei..."


In [ ]:
%%bigquery
alter table magazine_recipes_stg.Nutrition add column health_rating integer;

Executing query with job ID: 98fcf46d-bec0-4344-81ee-8d9fc57a1dc9
Query executing: 0.48s


ERROR:
 400 Column already exists: health_rating at [1:55]

Location: US
Job ID: 98fcf46d-bec0-4344-81ee-8d9fc57a1dc9



In [ ]:
%%bigquery
update magazine_recipes_stg.Nutrition set health_rating =
  (select CAST(json_value(ml_generate_text_llm_result, '$.rating') as INT64)
   from magazine_recipes_stg_ai.nutrition_ratings_10
   where recipe_id = CAST(json_value(ml_generate_text_llm_result, '$.recipe_id') as INT64))
where 1=1;


Query is running:   0%|          |

""


In [ ]:
%%bigquery
select count(*)
from magazine_recipes_stg.Nutrition
where health_rating is not null

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,19


#### Apply at larger scale

In [ ]:
%%bigquery
declare prompt_query STRING default "Rate the nutritional value as: 1,2 or 3. 1 being unhealthy, 2 being neutral, and 3 being  healthy. cholestrol above 36 is unhealthy, sodium above 341 is unhealthy, total fat above 18 is unhealthy, carbo above 26 is unhealthy, protein below 12 is unhealthy, iron below 2 is unhealthy, vitamin_c below 23 is unhealthy, vitamin_a below 1721 is unhealthy, fiber below 1 is unhealthy. If less than 5 metrics are unhealthy, the rating is 3. If 5 metrics are unhealthy, the rating is 2. If more than 5 metrics are unhealthy, the rating is 1. Return recipe_id, title, rating, nutritional values. Rating must not be null. Rating must be either 1,2, or 3";
create or replace table magazine_recipes_stg_ai.nutrition_ratings as
  select *
  from ML.generate_text(
    model remote_models.gemini_pro,
    (
      select concat(prompt_query, to_json_string(json_object("recipe_id", n.recipe_id, "title", title,
                    "carbo", carbo, "total_fat", total_fat, "cholestrl", cholestrl, "sodium", sodium, "calories",calories, "iron", iron, "vitamin_c", vitamin_c, "vitamin_a", vitamin_a))) as prompt
      from magazine_recipes_stg.Nutrition n
      inner join magazine_recipes_stg.Recipes r on r.recipe_id = n.recipe_id
    ),
    struct(TRUE as flatten_json_output)
  );

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select ml_generate_text_llm_result, prompt
from magazine_recipes_stg_ai.nutrition_ratings

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,prompt
0,"{""recipe_id"":477,""title"":""Red Onion Potato Sal...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
1,"{""calories"":92.43,""carbo"":17.67,""cholestrl"":27...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
2,"{""calories"":310.86,""carbo"":34.21,""cholestrl"":3...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
3,"{""calories"":133.52,""carbo"":12.43,""cholestrl"":2...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
4,"{""calories"":697.84,""carbo"":90.24,""cholestrl"":3...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
...,...,...
873,"{""calories"":134.82,""carbo"":22.4,""cholestrl"":51...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
874,"{""calories"":570.94,""carbo"":53.85,""cholestrl"":1...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
875,"{""calories"":249.69,""carbo"":52.92,""cholestrl"":0...","Rate the nutritional value as: 1,2 or 3. 1 bei..."
876,"{""calories"":55.74,""carbo"":13.14,""cholestrl"":0....","Rate the nutritional value as: 1,2 or 3. 1 bei..."


In [ ]:
%%bigquery
update magazine_recipes_stg.Nutrition set health_rating =
  (select CAST(json_value(ml_generate_text_llm_result, '$.rating') as INT64)
   from magazine_recipes_stg_ai.nutrition_ratings
   where recipe_id = CAST(json_value(ml_generate_text_llm_result, '$.recipe_id') as INT64))
where 1=1;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select count(*)
from magazine_recipes_stg.Nutrition
where health_rating is NULL

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,5


In [ ]:
# it appears that the AI was not able to determine a rating for 5 recipes despite the prompt, we will leave those as null

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select health_rating, count(*) from magazine_recipes_stg.Nutrition
group by health_rating

Query is running:   0%|          |

Downloading:   0%|          |

,health_rating,f0_
0,<NA>,5
1,1,88
2,2,287
3,3,498


In [ ]:
%%bigquery
create or replace table magazine_recipes_stg.Nutrition as
select * except(data_source, load_time), data_source, load_time
from  magazine_recipes_stg.Nutrition

Query is running:   0%|          |

""


In [ ]:
# update the datasource for recipes with health rating
%%bigquery
update magazine_recipes_stg.Nutrition
set data_source = 'bird-ai'
where health_rating is not null

Query is running:   0%|          |

""


# Delta Detection

In [ ]:
'''tables that were changed and what changes were done:
- recipes (type was removed- records updated)
- nutrition (added heatlh score - records updated)
- ingredients (airtable and bird ingreidents condensed- records were removed and records were updated)
non delta detection but create/deletion of tables
- magazines, publications, and journalists : tables were deleted
- posts: table was created
'''

In [ ]:
# for ingredients
%%bigquery
select count(*) as num_updates
from magazine_recipes_csp.Ingredients t full join magazine_recipes_stg.Ingredients s
on t.ingredient_id = s.ingredient_id
where t.status_flag = true
and (s.ingredient_id is null and t.ingredient_id is not null
or s.data_source != t.data_source);

Query is running:   0%|          |

Downloading:   0%|          |

,num_updates
0,26


In [ ]:
# process the deletes from ingredients
%%bigquery
declare current_ts TIMESTAMP;
set current_ts = current_timestamp();
update magazine_recipes_csp.Ingredients
set discontinue_time = timestamp_sub(current_ts, interval 1 second), status_flag = false
where ingredient_id in (select t.ingredient_id
from magazine_recipes_csp.Ingredients t left join magazine_recipes_stg.Ingredients s
on t.ingredient_id = s.ingredient_id
where s.ingredient_id is null);

Query is running:   0%|          |

""


In [ ]:
# for ingredients
%%bigquery
select count(*) as num_updates
from magazine_recipes_csp.Ingredients t full join magazine_recipes_stg.Ingredients s
on t.ingredient_id = s.ingredient_id
where t.status_flag = true
and (s.ingredient_id is null and t.ingredient_id is not null
or s.data_source != t.data_source);

Query is running:   0%|          |

Downloading:   0%|          |

,num_updates
0,7


In [ ]:
# process the updates from ingredients
%%bigquery
declare current_ts TIMESTAMP;
set current_ts = current_timestamp();

create temp table updates as
  select s.*
  from magazine_recipes_csp.Ingredients t full join magazine_recipes_stg.Ingredients s
  on t.ingredient_id = s.ingredient_id
  and s.data_source != t.data_source
  where s.ingredient_id is not null;

update magazine_recipes_csp.Ingredients
set discontinue_time = timestamp_sub(current_ts, interval 1 second), status_flag = false
where ingredient_id in (select ingredient_id from updates);

insert into magazine_recipes_csp.Ingredients
  (ingredient_id, category, name, plural, data_source, load_time, effective_time, status_flag)
    (select ingredient_id, category, name, plural, data_source, load_time, current_ts, true
      from updates);

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select count(*)
from magazine_recipes_csp.Ingredients
where status_flag = True

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,3350


In [ ]:
%%bigquery
select count(*)
from magazine_recipes_stg.Ingredients


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,3350


In [ ]:
# Nutrition CDC
%%bigquery
alter table magazine_recipes_csp.Nutrition
  ADD column health_rating INT64;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select count(*) as num_records
from magazine_recipes_csp.Nutrition

Query is running:   0%|          |

Downloading:   0%|          |

,num_records
0,878


In [ ]:
%%bigquery
select count(*) as num_updates
from magazine_recipes_stg.Nutrition s join magazine_recipes_csp.Nutrition t
on s.recipe_id = t.recipe_id
where t.status_flag = true
and (s.health_rating is not null and t.health_rating is null)

Query is running:   0%|          |

Downloading:   0%|          |

,num_updates
0,873


In [ ]:
  %%bigquery
  select s.*
  from magazine_recipes_csp.Nutrition t join magazine_recipes_stg.Nutrition s
  on s.recipe_id = t.recipe_id
  WHERE (s.health_rating is not null and t.health_rating is null) AND t.protien is NULL ;

Query is running:   0%|          |

Downloading: |          |

,recipe_id,protien,carbo,alcohol,total_fat,sat_fat,cholestrl,sodium,iron,vitamin_c,vitamin_a,fiber,pcnt_cal_carb,pcnt_cal_fat,pcnt_cal_prot,calories,health_rating,data_source,load_time


In [ ]:
%%bigquery
declare current_ts TIMESTAMP;
set current_ts = current_timestamp();

create temp table updates as
  select s.*
  from magazine_recipes_csp.Nutrition t join magazine_recipes_stg.Nutrition s
  on s.recipe_id = t.recipe_id
  WHERE (s.health_rating is not null and t.health_rating is null);

update magazine_recipes_csp.Nutrition
set discontinue_time = timestamp_sub(current_ts, interval 1 second), status_flag = false
where recipe_id in (select recipe_id from updates);

insert into magazine_recipes_csp.Nutrition
  (recipe_id, protien, carbo, alcohol, total_fat, sat_fat, cholestrl, sodium, iron, vitamin_c, vitamin_a, fiber, pcnt_cal_carb, pcnt_cal_fat, pcnt_cal_prot, calories, health_rating, data_source, load_time, effective_time, status_flag)
    (select recipe_id, protien, carbo, alcohol, total_fat, sat_fat, cholestrl, sodium, iron, vitamin_c, vitamin_a, fiber, pcnt_cal_carb, pcnt_cal_fat, pcnt_cal_prot, calories, health_rating, data_source, load_time, current_ts, true
        from updates);

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select count(*) as num_updates
from magazine_recipes_stg.Nutrition s join magazine_recipes_csp.Nutrition t
on s.recipe_id = t.recipe_id
where t.status_flag = true
AND (s.health_rating != t.health_rating )

Query is running:   0%|          |

Downloading:   0%|          |

,num_updates
0,0


## Recipes CDC

In [ ]:
%%bigquery
select count(*)
from magazine_recipes_csp.Recipes
where status_flag = false

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,0


In [ ]:
# discontinue records from consumption not in staging
%%bigquery
update magazine_recipes_csp.Recipes
  set status_flag = FALSE, discontinue_time = timestamp_sub(current_timestamp(), interval 1 second)
  where recipe_id in (select t.recipe_id  from magazine_recipes_csp.Recipes t
                      left join magazine_recipes_stg.Recipes s
                      on t.recipe_id = s.recipe_id
                      where t.status_flag = true
                      and s.recipe_id is null);


Query is running:   0%|          |

""


In [ ]:
%%bigquery
select count(*)
from magazine_recipes_csp.Recipes
where status_flag = false

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,153


In [ ]:
# insert records without timestamps
%%bigquery
declare current_ts TIMESTAMP;
set current_ts = current_timestamp();
insert into magazine_recipes_csp.Recipes(recipe_id, title, subtitle, servings, yield_unit, prep_min, cook_min, stnd_min, source, intro, directions, rating, ease_of_prep, note, page, slowcooker, link, last_made, data_source, load_time, effective_time, status_flag)
    (select recipe_id, title, subtitle, servings, yield_unit, prep_min, cook_min, stnd_min, source, intro, directions, rating, ease_of_prep, note, page, slowcooker, link, last_made, data_source, load_time, current_ts, true
        from (select s.recipe_id, s.title, s.subtitle, s.servings, s.yield_unit, s.prep_min, s.cook_min, s.stnd_min, s.source, s.intro, s.directions, s.rating, s.ease_of_prep, s.note, s.page, s.slowcooker, s.link, s.last_made, s.data_source, s.load_time
            from magazine_recipes_csp.Recipes t
                      join magazine_recipes_stg.Recipes s
                      on t.recipe_id = s.recipe_id
                      where t.status_flag = true
                      and t.type is not null));

Query is running:   0%|          |

""


In [ ]:
# discontinue records with timestamps
%%bigquery
update magazine_recipes_csp.Recipes
  set status_flag = FALSE, discontinue_time = timestamp_sub(current_timestamp(), interval 1 second)
  where type is not null;


Query is running:   0%|          |

""


In [ ]:
%%bigquery
select count(*)
from magazine_recipes_csp.Recipes
where status_flag = True

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,1023


In [ ]:
%%bigquery
select count(*)
from magazine_recipes_stg.Recipes

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,1023


## Magazine, Journalist, and Publication CDC

In [ ]:
%%bigquery
update magazine_recipes_csp.Magazines
  set status_flag = FALSE, discontinue_time = timestamp_sub(current_timestamp(), interval 1 second)
  where 1 = 1;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
update magazine_recipes_csp.Journalists
  set status_flag = FALSE, discontinue_time = timestamp_sub(current_timestamp(), interval 1 second)
  where 1 = 1;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
update magazine_recipes_csp.Publications
  set status_flag = FALSE, discontinue_time = timestamp_sub(current_timestamp(), interval 1 second)
  where 1 = 1;

Query is running:   0%|          |

""


## Add Posts and Users to Consumption

In [ ]:
%%bigquery
create or replace table magazine_recipes_csp.Users(
  user_id	INT64 not null,
  username STRING not null,
  f_name STRING not null,
  l_name STRING not null,
  age INT64 not null,
  phone STRING not null,
  state STRING not null,
  data_source STRING not null,
  load_time	TIMESTAMP not null,
  effective_time TIMESTAMP default current_timestamp() not null,
  discontinue_time TIMESTAMP,
  status_flag BOOL not null,
  primary key (user_id, effective_time) not enforced)
  as select *, current_timestamp(), null, true
  from magazine_recipes_stg.Users;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
create or replace table magazine_recipes_csp.Posts(
  post_id INT64 not null,
  recipe_id INT64 not null,
  user_id INT64 not null,
  page STRING not null,
  timestamp_posted TIMESTAMP not null,
  data_source STRING not null,
  load_time	TIMESTAMP not null,
  effective_time TIMESTAMP default current_timestamp() not null,
  discontinue_time TIMESTAMP,
  status_flag BOOL not null,
  primary key (post_id, effective_time) not enforced)
  as select *, current_timestamp(), null, true
  from magazine_recipes_stg.Posts;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT * FROM shidcs329e.magazine_recipes_stg.__TABLES__


Query is running:   0%|          |

Downloading:   0%|          |

,project_id,dataset_id,table_id,creation_time,last_modified_time,row_count,size_bytes,type
0,shidcs329e,magazine_recipes_stg,Ingredients,1707520137650,1712624321752,3350,190911,1
1,shidcs329e,magazine_recipes_stg,Ingredients_duplicates,1712371310673,1712371310674,8,266,1
2,shidcs329e,magazine_recipes_stg,Journalists,1711745811428,1711745892878,91,6242,1
3,shidcs329e,magazine_recipes_stg,Journalists_copy,1711745794545,1711745794545,90,6193,1
4,shidcs329e,magazine_recipes_stg,Magazines,1707525393272,1707529056377,20,320,1
5,shidcs329e,magazine_recipes_stg,Nutrition,1712620361000,1712620405472,878,134279,1
6,shidcs329e,magazine_recipes_stg,Posts,1712619159362,1712619159363,1023,55916,1
7,shidcs329e,magazine_recipes_stg,Publications,1707527111554,1712349014421,1176,47040,1
8,shidcs329e,magazine_recipes_stg,Quantity,1707517223718,1712376936432,5463,375634,1
9,shidcs329e,magazine_recipes_stg,Recipes,1712620584232,1712620584233,1023,463849,1


In [ ]:
%%bigquery
drop table magazine_recipes_stg.recipe_ingredient_at;
drop table magazine_recipes_stg.quantity_at;
drop table magazine_recipes_stg.ingredients_trial;
drop table magazine_recipes_stg.ingredients_at;
drop table magazine_recipes_stg.ingredients;
drop table magazine_recipes_stg.ingredient_id_no_nulls;
drop table magazine_recipes_stg.Journalists_copy;
drop table magazine_recipes_stg.Ingredients_duplicates;
drop table magazine_recipes_stg.Journalists;
drop table magazine_recipes_stg.Magazines;
drop table magazine_recipes_stg.Publications;


Query is running:   0%|          |

""


In [ ]:
%%bigquery
drop table magazine_recipes_stg.unique_ingredient_with_id;
drop table magazine_recipes_stg.type_predictions_raw_10;


Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT * FROM shidcs329e.magazine_recipes_stg.__TABLES__

Query is running:   0%|          |

Downloading:   0%|          |

,project_id,dataset_id,table_id,creation_time,last_modified_time,row_count,size_bytes,type
0,shidcs329e,magazine_recipes_stg,Ingredients,1707520137650,1712624321752,3350,190911,1
1,shidcs329e,magazine_recipes_stg,Nutrition,1712620361000,1712620405472,878,134279,1
2,shidcs329e,magazine_recipes_stg,Posts,1712619159362,1712619159363,1023,55916,1
3,shidcs329e,magazine_recipes_stg,Quantity,1707517223718,1712376936432,5463,375634,1
4,shidcs329e,magazine_recipes_stg,Recipes,1712620584232,1712620584233,1023,463849,1
5,shidcs329e,magazine_recipes_stg,Users,1712619450589,1712619450590,91,7921,1


# Enforcing PK and FK

In [ ]:
# Add primary key for Recipes
%%bigquery
alter table magazine_recipes_stg.Recipes
  add primary key (recipe_id) not enforced;

Query is running:   0%|          |

""


In [ ]:
# Add primary key for nutrition
%%bigquery
alter table magazine_recipes_stg.Nutrition
  add primary key (recipe_id) not enforced;

Query is running:   0%|          |

""


In [ ]:
# Foreign Key for  Nutrition
%%bigquery
alter table magazine_recipes_stg.Nutrition add foreign key (recipe_id)
  references magazine_recipes_stg.Recipes (recipe_id) not enforced

Query is running:   0%|          |

""


In [12]:
# Primary key for posts
%%bigquery
alter table magazine_recipes_stg.Posts
  add primary key (post_id) not enforced;

Query is running:   0%|          |

""


In [ ]:
# Primary Key for Users
%%bigquery
alter table magazine_recipes_stg.Users
  add primary key (user_id) not enforced;

Query is running:   0%|          |

""


In [13]:
# Foreign Key for Posts
%%bigquery
alter table magazine_recipes_stg.Posts add foreign key (recipe_id)
  references magazine_recipes_stg.Recipes (recipe_id) not enforced;

Query is running:   0%|          |

""


In [14]:
# user_id Foreign Key for Posts
%%bigquery
alter table magazine_recipes_stg.Posts add foreign key (user_id)
  references magazine_recipes_stg.Users (user_id) not enforced;

Query is running:   0%|          |

""


In [ ]:
# Recipe_id foreign key for quantity
%%bigquery
alter table magazine_recipes_stg.Quantity add foreign key (recipe_id)
  references magazine_recipes_stg.Recipes (recipe_id) not enforced;

Query is running:   0%|          |

""


## Checking Primary keys

In [ ]:
# There are no duplicates in stg or csp for Users (We just made Users so there shouldn't be any updates reflected in csp )
%%bigquery
SELECT user_id, count(*) from magazine_recipes_stg.Users
GROUP BY user_id
HAVING count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,user_id,f0_


In [ ]:
# Checking Recipes for duplicate PKs
%%bigquery
SELECT recipe_id, count(*) from magazine_recipes_stg.Recipes
GROUP BY recipe_id
HAVING count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,recipe_id,f0_


In [ ]:
# We deleted Journalists from stg so let's just confirm that the csp has the right amount of updates
# The query returns three results as expected because we changed three existing journalists in p8
%%bigquery
SELECT journalist_id, count(*) from magazine_recipes_csp.Journalists
GROUP BY journalist_id
HAVING count(*) > 1

Query is running:   0%|          |

Downloading:   0%|          |

,journalist_id,f0_
0,15,2
1,3,2
2,73,2


In [ ]:
# This query returns no results as expected because between p8 and p9, no publication_ids have been altered
# furthermore this table will no longer be active which is hwy we're checking csp instead of stg
%%bigquery
SELECT publication_id, count(*) from magazine_recipes_csp.Publications
GROUP BY publication_id
HAVING count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,publication_id,f0_


In [ ]:
# Returns zero results - This also means there are no duplicated PKs in csp either
%%bigquery
select post_id, count(*) from magazine_recipes_stg.Posts
GROUP BY post_id
HAVING count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,post_id,f0_


In [ ]:
# Check duplicate PKs in ingredients - there are none!
# In csp there are two of almost every id because we updated all but maybe 7 to 14 ingredients in p9
%%bigquery
SELECT ingredient_id, count(*) from magazine_recipes_stg.Ingredients
GROUP BY ingredient_id
HAVING count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,ingredient_id,f0_


In [ ]:
# Check duplicate PKs in Nutrition
%%bigquery
SELECT recipe_id, count(*) from magazine_recipes_stg.Nutrition
GROUP BY recipe_id
HAVING count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,recipe_id,f0_


## Checking Foreign Keys for New Tables

In [ ]:
%%bigquery
select count(*) as orphan_records
from magazine_recipes_stg.Posts
where recipe_id not in (select recipe_id from  magazine_recipes_stg.Recipes)

Query is running:   0%|          |

Downloading:   0%|          |

,orphan_records
0,0


In [ ]:
# Orphan records for Quantity
%%bigquery
select count(*) as orphan_records
from magazine_recipes_stg.Quantity
where recipe_id not in (select recipe_id from  magazine_recipes_stg.Recipes)

Query is running:   0%|          |

Downloading:   0%|          |

,orphan_records
0,0


In [ ]:
# Orpahn Records for Nutrition
%%bigquery
select count(*) as orphan_records
from magazine_recipes_stg.Nutrition
where recipe_id not in (select recipe_id from  magazine_recipes_stg.Recipes)

Query is running:   0%|          |

Downloading:   0%|          |

,orphan_records
0,0


## Fixing Orphan Records for Posts

In P8 we deleted some Users but we never fixed the posts that were associated with them. Here we'll reassign the correct post_id to the orphan posts

In [ ]:
%%bigquery
SELECT *
FROM magazine_recipes_stg.Users
WHERE user_id > 90

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,username,f_name,l_name,age,phone,state,data_source,load_time
0,92,ChefJimmy92,Jimmy,Falon,22,214-417-1738,HI,faker-ai,2024-03-29 20:34:11.191045+00:00
1,93,MattsMarvelousMeals,Matthew,McConaughey,33,972-732-6574,WI,faker-ai,2024-03-29 20:34:11.191045+00:00
2,94,TrishaPieQueen94,Trisha,Paytas,40,817-370-7063,CA,faker-ai,2024-03-29 20:34:11.191045+00:00
3,91,TaylorsTastyTreats,Taylor,Swift,64,682-270-4005,NY,faker-ai,2024-03-29 20:34:11.191045+00:00


In [6]:
%%bigquery
CREATE OR REPLACE TABLE magazine_recipes_csp.Posts_copy AS
SELECT
  post_id,
  recipe_id,
  CASE
    WHEN user_id = 88 THEN 91
    WHEN user_id = 89 THEN 92
    WHEN user_id = 90 THEN 93
    ELSE user_id
  END AS user_id,
  page,
  timestamp_posted,
  data_source,
  load_time,
  effective_time,
  discontinue_time,
  status_flag
FROM
  magazine_recipes_csp.Posts

Query is running:   0%|          |

""


In [1]:
%%bigquery
CREATE OR REPLACE TABLE magazine_recipes_stg.Posts_copy AS
SELECT
  post_id,
  recipe_id,
  CASE
    WHEN user_id = 88 THEN 91
    WHEN user_id = 89 THEN 92
    WHEN user_id = 90 THEN 93
    ELSE user_id
  END AS user_id,
  page,
  timestamp_posted,
  datasource,
  load_time
FROM
  magazine_recipes_stg.Posts

Query is running:   0%|          |

""


In [10]:
%%bigquery
DROP TABLE magazine_recipes_csp.Posts;
drop table magazine_recipes_stg.Posts;

Query is running:   0%|          |

""


In [11]:
%%bigquery
ALTER TABLE magazine_recipes_csp.Posts_copy RENAME TO Posts;
ALTER TABLE magazine_recipes_stg.Posts_copy RENAME TO Posts;

Query is running:   0%|          |

""


In [16]:
# FIXED !!
%%bigquery
select count(*) as orphan_records
from magazine_recipes_stg.Posts
where user_id not in (select user_id from  magazine_recipes_stg.Users)

Query is running:   0%|          |

Downloading:   0%|          |

,orphan_records
0,0


In [18]:
%%bigquery
select count(*) as orphan_records
from magazine_recipes_stg.Quantity
where recipe_id not in (select recipe_id from  magazine_recipes_stg.Recipes)

Query is running:   0%|          |

Downloading:   0%|          |

,orphan_records
0,0


In [20]:
%%bigquery
select count(*) as orphan_records
from magazine_recipes_stg.Quantity
where ingredient_id not in (select ingredient_id from  magazine_recipes_stg.Ingredients)

Query is running:   0%|          |

Downloading:   0%|          |

,orphan_records
0,0


### End Project 9 Work

In [ ]:
# as a fun final outtake, we will share the odes the AI came up with when we prompted it to have fun!
%%bigquery
DECLARE prompt_query STRING DEFAULT "have fun with it!";
SELECT *
FROM ML.generate_text(
  MODEL remote_models.gemini_pro,
  (
    SELECT CONCAT(prompt_query, TO_JSON_STRING(JSON_OBJECT("ingredient_id", ingredient_id, "ingredient_name", name))) AS prompt
    FROM magazine_recipes_stg.Ingredients
    WHERE data_source = 'Airtable'
    ORDER BY ingredient_id
  )
);


Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_result,ml_generate_text_status,prompt
0,"{""candidates"":[{""content"":{""parts"":[{""text"":""*...",,"have fun with it!{""ingredient_id"":4646,""ingred..."
1,"{""candidates"":[{""content"":{""parts"":[{""text"":""*...",,"have fun with it!{""ingredient_id"":4647,""ingred..."
2,"{""candidates"":[{""content"":{""parts"":[{""text"":""*...",,"have fun with it!{""ingredient_id"":4649,""ingred..."
3,"{""candidates"":[{""content"":{""parts"":[{""text"":""*...",,"have fun with it!{""ingredient_id"":4650,""ingred..."
4,"{""candidates"":[{""content"":{""parts"":[{""text"":""*...",,"have fun with it!{""ingredient_id"":4652,""ingred..."
5,"{""candidates"":[{""content"":{""parts"":[{""text"":""*...",,"have fun with it!{""ingredient_id"":4653,""ingred..."
6,"{""candidates"":[{""content"":{""parts"":[{""text"":""*...",,"have fun with it!{""ingredient_id"":4654,""ingred..."
7,"{""candidates"":[{""content"":{""parts"":[{""text"":""*...",,"have fun with it!{""ingredient_id"":4655,""ingred..."
8,"{""candidates"":[{""content"":{""parts"":[{""text"":""*...",,"have fun with it!{""ingredient_id"":4657,""ingred..."
9,"{""candidates"":[{""content"":{""parts"":[{""text"":""*...",,"have fun with it!{""ingredient_id"":4659,""ingred..."
